# Week 2: Semantic Enrichment & Vector Search
## Adding Intelligence to Your Code Graph

In this notebook, we'll:
1. Load the Week 1 structural graph
2. Add LLM-generated summaries
3. Create vector embeddings
4. Build FAISS index
5. Test semantic search

In [ ]:
# Setup
import sys
from pathlib import Path
import pickle
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pandas as pd

sys.path.insert(0, '../src')

from utils.llm_client import LangChainClient
from indexing.semantic_enrichment import SemanticEnricher
from indexing.vector_store import VectorStore

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Load Week 1 Graph

In [ ]:
# Load the structural graph from Week 1
graph_path = '../data/graphs/code_graph.pkl'

with open(graph_path, 'rb') as f:
    graph = pickle.load(f)

print(f"Loaded graph: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")

In [ ]:
# Inspect a sample node BEFORE enrichment
sample_node = list(graph.nodes(data=True))[5]
node_id, attrs = sample_node

print(f"Node: {attrs['name']} ({attrs['type']})")
print(f"\nCurrent attributes:")
for key, value in attrs.items():
    if key != 'code':
        print(f"  {key}: {value}")
    else:
        print(f"  code: {len(value)} chars")

## 2. Semantic Enrichment with LLM

In [ ]:
# Initialize LLM client
llm_client = LangChainClient(
    provider='google',  # or 'openai', 'anthropic'
    model_name='gemini-2.0-flash-exp'
)

print(f"✓ LLM client initialized: {llm_client.model_name}")

In [ ]:
# Test enrichment on a single node first
enricher = SemanticEnricher(llm_client=llm_client)

# Pick a function node
test_node = None
for node_id, attrs in graph.nodes(data=True):
    if attrs.get('type') == 'function' and len(attrs.get('code', '')) > 50:
        test_node = (node_id, attrs)
        break

if test_node:
    node_id, attrs = test_node
    print(f"Testing enrichment on: {attrs['name']}\n")
    
    enriched = enricher.enrich_single(node_id, attrs)
    
    print("✓ Enrichment result:")
    print(f"  Summary: {enriched.summary}")
    print(f"  Description: {enriched.description}")
    print(f"  Tags: {', '.join(enriched.tags)}")
    print(f"  Complexity: {enriched.complexity}")
    print(f"  Purpose: {enriched.purpose}")

In [ ]:
# Enrich the entire graph (this will take a few minutes)
print("Enriching all nodes... (this may take 2-5 minutes)\n")

enriched_graph = enricher.enrich_graph(graph, skip_existing=True)

print("\n✓ Enrichment complete!")

In [ ]:
# Show enrichment statistics
stats = enricher.get_enrichment_stats(enriched_graph)

print(f"\n=== Enrichment Statistics ===")
print(f"Total nodes: {stats['total_nodes']}")
print(f"Enriched: {stats['enriched_nodes']} ({stats['enrichment_rate']:.1%})")
print(f"Unique tags: {stats['total_unique_tags']}")

print(f"\nTop tags:")
for tag, count in stats['top_tags'][:10]:
    print(f"  {tag}: {count}")

In [ ]:
# Visualize tag distribution
tag_data = stats['top_tags'][:15]
tags, counts = zip(*tag_data)

plt.figure(figsize=(12, 6))
plt.barh(tags, counts, color='skyblue')
plt.xlabel('Count')
plt.title('Top 15 Code Tags')
plt.tight_layout()
plt.show()

In [ ]:
# LLM caching enabled
print("\n=== LLM Configuration ===")
print(f"Provider: {llm_client.provider}")
print(f"Model: {llm_client.model_name}")
print(f"Cache: Enabled (LangChain InMemoryCache)")
print(f"Note: LLM API usage tracked automatically by the provider")

## 3. Build Vector Index

In [ ]:
# Initialize vector store
vector_store = VectorStore(
    provider='huggingface',  # Local, free, fast
    model_name='all-MiniLM-L6-v2',  # Fast and good quality
    index_type='Flat'   # Exact search
)

print(f"Vector store initialized:")
print(f"  Embedding dim: {vector_store.embedding_dim}")
print(f"  Index type: {vector_store.index_type}")

In [ ]:
# Build index from enriched graph
print("Building vector index...\n")

vector_store.build_from_graph(
    enriched_graph,
    text_field='combined',  # Combines code, summary, tags
    batch_size=32
)

print(f"\n✓ Vector index built: {len(vector_store.node_ids)} vectors")

## 4. Test Semantic Search

In [ ]:
def display_search_results(query, results):
    """Pretty print search results"""
    print(f"\n{'='*80}")
    print(f"Query: '{query}'")
    print(f"{'='*80}\n")
    
    for i, result in enumerate(results, 1):
        print(f"{i}. {result.name} ({result.type})")
        print(f"   Score: {result.score:.3f}")
        if result.summary:
            print(f"   Summary: {result.summary}")
        if result.tags:
            print(f"   Tags: {', '.join(result.tags[:5])}")
        print()

In [ ]:
# Test various queries
test_queries = [
    "function that validates user input",
    "code for reading and parsing data",
    "database connection or storage",
    "configuration and settings"
]

for query in test_queries:
    results = vector_store.search(query, top_k=3)
    display_search_results(query, results)

In [ ]:
# Interactive search
def interactive_search():
    print("\n🔍 Enter queries (or 'quit' to exit):\n")
    
    while True:
        query = input("Query > ").strip()
        
        if query.lower() in ['quit', 'exit', 'q']:
            break
        
        if not query:
            continue
        
        results = vector_store.search(query, top_k=5)
        display_search_results(query, results)

# Uncomment to use:
# interactive_search()

## 5. Analyze Search Quality

In [ ]:
# Compare score distributions
sample_queries = [
    "validation function",
    "data processing",
    "configuration management"
]

all_scores = []
query_labels = []

for query in sample_queries:
    results = vector_store.search(query, top_k=10)
    scores = [r.score for r in results]
    all_scores.extend(scores)
    query_labels.extend([query[:20]] * len(scores))

# Plot
df = pd.DataFrame({'Query': query_labels, 'Score': all_scores})

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Query', y='Score')
plt.title('Search Score Distribution by Query Type')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Find similar nodes
def find_and_display_similar(node_name, top_k=5):
    """Find nodes similar to a given node"""
    
    # Find node ID
    target_node_id = None
    for node_id, attrs in enriched_graph.nodes(data=True):
        if attrs.get('name') == node_name:
            target_node_id = node_id
            print(f"Found: {attrs['name']} ({attrs['type']})")
            print(f"Summary: {attrs.get('summary', 'N/A')}\n")
            break
    
    if not target_node_id:
        print(f"Node '{node_name}' not found")
        return
    
    # Find similar
    similar = vector_store.find_similar_nodes(target_node_id, top_k=top_k)
    
    print(f"Similar nodes:\n")
    for i, result in enumerate(similar, 1):
        print(f"{i}. {result.name} ({result.type}) - Score: {result.score:.3f}")
        if result.summary:
            print(f"   {result.summary}")
        print()

# Try it
find_and_display_similar('save_to_file', top_k=3)

## 6. Save Everything

In [ ]:
# Save enriched graph
enriched_path = '../data/graphs/code_graph_enriched.pkl'
with open(enriched_path, 'wb') as f:
    pickle.dump(enriched_graph, f)
print(f"✓ Enriched graph saved: {enriched_path}")

# Save vector store
vector_path = '../data/graphs/vector_store'
vector_store.save(vector_path)
print(f"✓ Vector store saved: {vector_path}")

## Summary

Week 2 Complete! You now have:
- ✅ LLM-generated summaries for all code entities
- ✅ Domain tags and complexity ratings
- ✅ Vector embeddings (384 dimensions)
- ✅ FAISS index for fast similarity search
- ✅ Semantic search capability

**Week 3 Preview:**
- Community detection (Louvain algorithm)
- Hierarchical summarization
- Global query support
- Query classification (global vs local)